节选自 Kaggle：MNIST 可被视为计算机视觉领域上的 "Hello World" 数据集。自 1999 年发布以来，这个经典的手写图像数据集一直是分类算法的基准，随着深度学习技术的发展，MNIST 依旧是研究人员和学习者的一个可靠资源。下图展示了该数据集的部分数据：
![MNIST-sample](assets/MnistExamples.png)]

该教程展示了：如何使用 PyTorch 进行简单的手写分类。部分代码参考了 PyTorch 官方教程内容 (BSD 3-Clause "New" or "Revised" License)。

在正式开始之前，我们对 PyTorch 进行安装。


In [ ]:
!pip install torch torchvision

首先，我们需要对 MNIST 数据集进行下载并创建数据加载器，但幸运的是 PyTorch 官方已经集成了下载方式。

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST  # 导入 MNIST 数据集类
from torchvision.transforms import Compose, ToTensor, Normalize  # 图像预处理

# 创建用于训练的 MNIST 数据集 和 加载器 (DataLoader)
train_mnist = MNIST(
    root='./', train=True, download=True,  # 若此前指定路径不存在 MNIST 数据，则自动下载
    transform=Compose([ToTensor(), Normalize((0.1307,), (0.3081,))]),  # 将 int[0, 255] 图像处理为 float32[0, 1] 并标准化
)
train_loader = DataLoader(train_mnist, batch_size=1024, shuffle=True)  # 设定一批数据由 1024 个图像构成，并打乱顺序抽取

# 创建用于测试的 MNIST 数据集 和 加载器 (DataLoader)
eval_mnist = MNIST(
    root='./', train=False, download=True,  # 与 Train 不同的是，这里 train 属性为 False
    transform=Compose([ToTensor(), Normalize((0.1307,), (0.3081,))]),
)
eval_loader = DataLoader(eval_mnist, batch_size=1024, shuffle=True)

接下来，我们将定义一个简单的网络用于对图像进行回归分类。对于不用的应用，网络应当进行对应的设计，但是对于 MNIST 数据集来说，如此简单的网络便足够。

In [ ]:
# nn 模块包含了 PyTorch 大部分的计算函数操作, Tensor 是 PyTorch 的基本计算单位-张量
from torch import nn, Tensor, max_pool2d, relu, dropout, log_softmax


class Net(nn.Module):

    # 初始化函数
    def __init__(self):
        super().__init__()  # 进行超类的继承

        # 图像特征提取
        self.conv_1 = nn.Conv2d(1, 16, kernel_size=5)  # 从图像中提取 16 维的特征
        self.conv_2 = nn.Conv2d(16, 32, kernel_size=5)  # 从特征中进一步提取 32 维的特征 这也是卷积神经网络的显著特征（连续多次卷积以提取更高维度特征）
        self.drop = nn.Dropout2d()  # 随机丢失部分前向传播计算 用于防止网络对数据过拟合

        # 利用高维度特征进行分类
        self.fc_1 = nn.Linear(512, 64)  # 输入维度计算 (in_c) = 特征维度 (32) * 特征图像大小 (16)
        self.fc_2 = nn.Linear(64, 10)  # 最后输出一个 10 维的数组，表示某张图片是 [0-9] 的概率

    # 前向传播所使用的函数
    def forward(self, x: Tensor) -> Tensor:
        # 第一次特征提取
        # [dim: 1, size: 28] -> [dim: 16, size: 24]
        x = self.conv_1(x)
        # [dim:16, size: 24] -> [dim: 16, size: 12]
        x = max_pool2d(x, kernel_size=2)
        # 激活函数 这里选择 ReLU (负值置零)
        x = relu(x)

        # 第二次特征提取
        # [dim: 16, size: 12] -> [dim: 32, size: 8]
        x = self.conv_2(x)
        # 随机丢弃 防止过拟合
        x = self.drop(x)
        # [dim: 32, size: 8] -> [dim: 32, size: 4]
        x = max_pool2d(x, kernel_size=2)
        # 激活函数 这里选择 ReLU (负值置零)
        x = relu(x)

        # 将特征展开并使用线性分类进一步提取特征。注：也有使用卷积直至形成 1x1 的例子，称为 Fully-Convolution 网络，这里不使用。
        # [dim: 32, size: 4] -> [dim: 512, size: 1]
        x = x.view(-1, 512)
        # [dim: 800, size: 1] -> [dim: 64, size: 1]
        x = self.fc_1(x)
        # 激活函数 这里选择 ReLU (负值置零)
        x = relu(x)
        # 随机丢弃 防止过拟合
        x = dropout(x, p=0.05, train=self.training)  # 丢弃的函数式写法，此处意为：当训练时，有5%的概率丢弃学习的数据

        # 根据特征进行最后的推断
        # [dim: 64, size: 1] -> [dim: 10, size: 1]
        x = self.fc_2(x)
        # 激活函数 这里选择 LogSoftmax (使和为 1 后进行 log 操作)
        # 相较于 Softmax，使用 LogSoftmax 可以更高地惩罚似然空间中更大的错误
        x = log_softmax(x, dim=1)

        # 返回各 [0-9] 网络预测的概率
        # 当使用 Softmax 激活时，若值为 [0.1, 0.2, 0.04, ...] 意味着网络认为：这个图像为 0 的概率为 0.1，为 1 的概率为 0.2，以此类推
        return x

这样，我们便有了一个可以将图像进行分类的 "人工智能" "深度学习" 网络。

接下来，我们将构建这个网络的训练流程，将按照：「输入 -> 网络 -> 输出 -> 计算损失函数 -> 梯度反向传播」 这个过程进行。

In [ ]:
from tqdm import tqdm
from torch.optim import SGD
from torch.nn.functional import nll_loss
import torch

# 创建一个 Net 的具体变量
net = Net()
# 若 CUDA 计算单元可用，将模型转移到 CUDA 中
if torch.cuda.is_available():
    net.cuda()

# 创建一个优化器，这里使用 SGD，是最为朴素的梯度下降策略
optimizer = SGD(net.parameters(), lr=0.01, momentum=0.5)


# 定义训练流程，epoch 表示当前是第 i 轮训练
def train_process(epoch: int):
    # 使网络进入训练模式，这将影响 BatchNorm, DropOut 等操作
    net.train()

    # 加载数据
    tqdm_process = tqdm(enumerate(train_loader))  # 进度条
    for batch_idx, (data, target) in tqdm_process:
        # 若 CUDA 计算单元可用，将数据转移到 CUDA 中
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()  # 此处使用 .cuda()，对于高级用户 PyTorch 提供了更具体的操作 .to()

        # 在一些古早的教程中，可能含有如下两行，是 PyTorch 0.4 的变量问题，目前已不再需要
        # data, target = Variable(data), Variable(target)

        # 前向传播 [size: 1024(batch_size), 1, 32, 32] -> net -> [size: 1024(batch_size), 10]
        pred = net(data)

        # 计算损失函数，这里选择 nll 损失
        loss = nll_loss(pred, target)

        # 反向传播
        optimizer.zero_grad()  # 清空优化器梯度
        loss.backward()  # 通过损失函数计算梯度
        optimizer.step()  # 使用优化器对网络参数进行优化

        # 打印损失函数 （每 16 组数据）
        tqdm_process.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()
            )
        )


# 定义测试流程
@torch.no_grad()  # 表示在这个函数中，禁用 autograd 的梯度跟踪
def eval_process():
    # 使网络进入测试模式
    net.eval()

    # 记录损失（无梯度）
    loss, correct = 0, 0

    # 加载数据
    tqdm_process = tqdm(eval_loader)  # 进度条
    for data, target in tqdm_process:
        # 若 CUDA 计算单元可用，将数据转移到 CUDA 中
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()  # 此处使用 .cuda()，对于高级用户 PyTorch 提供了更具体的操作 .to()

        # 在一些古早的教程中，可能含有如下两行，是 PyTorch 0.4 的变量问题，目前已不再需要
        # data, target = Variable(data, volatile=True), Variable(target)

        # 前向传播 [size: 1024(batch_size), 1, 32, 32] -> net -> [size: 1024(batch_size), 10]
        pred = net(data)

        # 计算损失函数，这里选择 nll 损失
        loss += nll_loss(pred, target, size_average=True)  # 对损失函数进行求和

        # 统计正确个数
        pred = pred.max(1, keepdim=True)[1]  # 获取具有最大可能性的那个数字
        correct += pred.eq(target.view_as(pred)).long().cpu().sum()

    # 展示损失与准确率
    loss /= len(eval_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            loss, correct, len(eval_loader.dataset),
            100. * correct / len(eval_loader.dataset)
        )
    )

至此，我们已经完成了一个简单深度学习的全部必要函数。接下来，我们将开始我们的训练。

In [ ]:
for epoch in range(1, 10 + 1):
    train_process(epoch)
    eval_process()